In [2]:
import polars as pl
import numpy as np
from datetime import datetime, timedelta

In [ ]:
np.random.seed(42)
def generate_coffee_data():
    n_records = 2000  # 2000 transactions over 6 months
    #Menue:
    menu_items = ['Espresso', 'Cappuccino', 'Latte', 'Americano', 'Mocha', 'Cold Brew']
    prices = [2.50, 4.00, 4.50, 3.00, 5.00, 3.50]
    # Getting dates over 6 months
    start_date = datetime(2023, 6, 1)
    dates = [start_date + timedelta(days=np.random.randint(0, 180))
            for _ in range(n_records)]
    data = {
        'date': dates,
        'drink': np.random.choice(menu_items, n_records),
        'price': [prices[menu_items.index(drink)] for drink in
                  np.random.choice(menu_items, n_records)],
        'quantity': np.random.choice([1, 1, 1, 2, 2, 3], n_records),  # Most people buy 1
        'customer_type': np.random.choice(['Regular', 'New', 'Tourist'],
                                        n_records, p=[0.5, 0.3, 0.2]),
        'payment_method': np.random.choice(['Card', 'Cash', 'Mobile'],
                                         n_records, p=[0.6, 0.2, 0.2]),
        'rating': np.random.choice([2,3, 4, 5], n_records, p=[0.1, 0.4, 0.4, 0.1])
    }
    return data
# Coffee shop DataFrame
coffee_data = generate_coffee_data()
df = pl.DataFrame(coffee_data)